In [1]:
import pandas as pd
import numpy as np

In [36]:
df_houses = pd.read_csv("data/King_County_House_prices_dataset.csv")

# Hypotheses

1. House prises vary by sqft, therefore the price/sqft is a better measure
1. The house price is depending on the area
1. Checking if prices differs by ZIP-codes 
1. Is the waterfront a positive factor?

1. The number of Bed- and bathrooms influences the price

# Checking the dataset

At the beginning it is recommended to check the shape of the dataset with `.shape`, `head()` and `info()` to get an overview of the different columns. The I will check the datatypes of the different columns and if they are suitable for further processing. Otherwise they have to be converted to a more suitable datatype.

In [39]:
df_houses.shape

(21597, 21)

the dataset has 21597 lines and 21 columns.

In [87]:
df_houses.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,NaN,0.0,...,7,1180,0.0,1955,0.0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,7,2170,400.0,1951,1991.0,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,6,770,0.0,1933,NaN,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,7,1050,910.0,1965,0.0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,8,1680,0.0,1987,0.0,98074,47.6168,-122.045,1800,7503


In [119]:
df_houses2 = df_houses.groupby("zipcode")
df_houses2.head()

AttributeError: 'NoneType' object has no attribute 'groupby'

In [86]:
df_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21597 non-null  int64         
 1   date           21597 non-null  datetime64[ns]
 2   price          21597 non-null  float64       
 3   bedrooms       21597 non-null  int64         
 4   bathrooms      21597 non-null  float64       
 5   sqft_living    21597 non-null  int64         
 6   sqft_lot       21597 non-null  int64         
 7   floors         21597 non-null  float64       
 8   waterfront     19221 non-null  float64       
 9   view           21534 non-null  float64       
 10  condition      21597 non-null  int64         
 11  grade          21597 non-null  int64         
 12  sqft_above     21597 non-null  int64         
 13  sqft_basement  21143 non-null  object        
 14  yr_built       21597 non-null  int64         
 15  yr_renovated   1775

The datatype of `date`is an object, that is not suitable for calculation. Therefore we go deeper into the analysis and find out that it's a string. It should be converted to a datetime, that can be used for calulations. As well the column `sqft_basement`should be converted to an integer.

In [85]:
# check the data type of the first entry
type(df_houses["date"][1])

pandas._libs.tslibs.timestamps.Timestamp

In [53]:
# change the datatype of "date" to datetime
df_houses["date"] = pd.to_datetime(df_houses["date"], format = "%m/%d/%Y")

In [71]:
df_houses["sqft_basement"].sort_values(ascending=False)

1127     990.0
8557     990.0
18332    990.0
6571     990.0
12268    990.0
         ...  
21442      NaN
21447      NaN
21473      NaN
21519      NaN
21581      NaN
Name: sqft_basement, Length: 21597, dtype: object

In [100]:
type(df_houses["sqft_basement"][0])

str

In [106]:
df_houses1 = pd.to_numeric(df_houses["sqft_basement"], downcast="integer")
type(df_houses1)

pandas.core.series.Series

In [98]:
#df.info()
df_houses["sqft_basement"] =df_houses.sqft_basement.replace("?", np.NaN)
df_houses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21597 entries, 0 to 21596
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21597 non-null  int64         
 1   date           21597 non-null  datetime64[ns]
 2   price          21597 non-null  float64       
 3   bedrooms       21597 non-null  int64         
 4   bathrooms      21597 non-null  float64       
 5   sqft_living    21597 non-null  int64         
 6   sqft_lot       21597 non-null  int64         
 7   floors         21597 non-null  float64       
 8   waterfront     19221 non-null  float64       
 9   view           21534 non-null  float64       
 10  condition      21597 non-null  int64         
 11  grade          21597 non-null  int64         
 12  sqft_above     21597 non-null  int64         
 13  sqft_basement  21143 non-null  object        
 14  yr_built       21597 non-null  int64         
 15  yr_renovated   1775

Furthermore it is interesting to investigate the number of duplicate rows, to drop the duplicates from the dataset. 
> In this data is no duplicate.

In [22]:
df.duplicated().value_counts()

False    21597
dtype: int64

# Checking the values

In [110]:
# Checking the values
df_houses.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.159700e+04,2.159700e+04,21597.000000,21597.000000,21597.000000,2.159700e+04,21597.000000,19221.000000,21534.000000,21597.000000,21597.000000,21597.000000,21597.000000,17755.000000,21597.000000,21597.000000,21597.000000,21597.000000,21597.000000
mean,4.580474e+09,5.402966e+05,3.373200,2.115826,2080.321850,1.509941e+04,1.494096,0.007596,0.233863,3.409825,7.657915,1788.596842,1970.999676,83.636778,98077.951845,47.560093,-122.213982,1986.620318,12758.283512
std,2.876736e+09,3.673681e+05,0.926299,0.768984,918.106125,4.141264e+04,0.539683,0.086825,0.765686,0.650546,1.173200,827.759761,29.375234,399.946414,53.513072,0.138552,0.140724,685.230472,27274.441950
min,1.000102e+06,7.800000e+04,1.000000,0.500000,370.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,3.000000,370.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.220000e+05,3.000000,1.750000,1430.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,1951.000000,0.000000,98033.000000,47.471100,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,1975.000000,0.000000,98065.000000,47.571800,-122.231000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068500e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


In [113]:
df1=df_houses.query["prices"] < 5000000

TypeError: 'method' object is not subscriptable

In [89]:
df_houses["price"].mean()

540296.5735055795

In [90]:
df_houses["price"].median()

450000.0

In [92]:
df_houses["price"].max()

7700000.0

In [94]:
df_houses["price"].std()

367368.1401013945

In [93]:
df_houses["price"].min()

78000.0

# Feature Engineering
Enrich the dataset with new columns to get new insights

In [118]:
# price per sqm
df_houses.assign("price_per_sqm") = price/sqft_living

NameError: name 'price' is not defined